In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tf_slim as slim
import tensorflow_datasets as tfds

In [3]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [ ]:
'train_img' = p'train-images-idx3-ubyte.gz'
'train_label':'train-labels-idx1-ubyte.gz'
'test_img':'t10k-images-idx3-ubyte.gz'
'test_label':'t10k-labels-idx1-ubyte.gz'

In [7]:
mnist_data, mnist_info = tfds.load('mnist', with_info=True, as_supervised=True)
mnist_data

2024-09-29 13:24:33.886244: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/sunohk/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


2024-09-29 13:24:37.596070: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-09-29 13:24:37.596100: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-09-29 13:24:37.596112: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-09-29 13:24:37.596193: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-29 13:24:37.596244: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


{'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [8]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='/Users/sunohk/tensorflow_datasets/mnist/3.0.1.incompleteNAONOP',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}


In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0  # Normalize to [0, 1] #픽셀값이 255이내 이므로
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0    # Normalize to [0, 1]
y_train = to_categorical(y_train, num_classes=10)  # One-hot encoding
y_test = to_categorical(y_test, num_classes=10)      # One-hot encoding

11490434/11490434 [==============================] - 1s 0us/step


In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Input layer
inputs = tf.keras.Input(shape=(28, 28, 1), name='x-in')

# Conv2D layer 1 
    #ReLu를 사용한 이유 : 
    # 1) 합성곱 층에서는 입력 데이터의 특징을 추출하는 것이 중요 -> ReLu는 비선형성을 제공하여 신경망이 더 복잡한 패턴을 학습할 수 있도록 함
    # 2) 양수 입력에 대한 기울기가 1이므로 학습 시 기울기 소실 문제(역전파 과정에서 출력층에서 멀어질수록 = 입력층과 가까울수록 기울기 값이 매우 작아지는 현상)를 줄여줌
    # 3) 계산 효율성 = ReLu는 계산이 간단하고 빠름
    # 4) 음수 값을 0으로 만들기 때문에 중요하지 않은 정보는 무시, 중요한 특징에 집중하게 함
conv1 = layers.Conv2D(5, (5, 5), activation='relu', padding='same')(inputs) #28*28
pool1 = layers.MaxPooling2D((2, 2))(conv1) #maxpooling 을 통해 2*2 크기로 축소

# Conv2D layer 2
conv2 = layers.Conv2D(5, (5, 5), activation='relu', padding='same')(pool1) #14*14
pool2 = layers.MaxPooling2D((2, 2))(conv2)

# Conv2D layer 3
conv3 = layers.Conv2D(20, (5, 5), activation='relu', padding='same')(pool2) #7*7
pool3 = layers.MaxPooling2D((2, 2))(conv3) #3*3 ( 7/2=3.*** 이므로 )

# Flatten layer
flat = layers.Flatten()(pool3) #1차원 배열로 변환 -> fully conncted layer로 연결할 수 있음 3*3*5 = 45 크기

# Fully connected layer with softmax activation 
outputs = layers.Dense(10, activation='softmax')(flat) 
    #dense : 완전연결층 , 10 = 0~9까지의 수를 분류하는 10개의 클래스
    #softmax : 주어진 입력 벡터를 확률 분포로 변환(0~1 사이 값으로 출력, 모든 출력 값의 합은 1) = 다중 클래스 분류 문제에서 각 클래스에 대한 확률 제공

# Create the model
model = models.Model(inputs=inputs, outputs=outputs)

# Use legacy Adam optimizer to avoid M1/M2 slowdown
optimizer = tf.keras.optimizers.legacy.Adam(1e-4) # adam 최적화기(optimizer) 사용

# Compile the model
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy', #다중 클래스(클래스 3개 이상) 분류 문제에서 사용되는 손실 함수(cross entropy : 예측과 실제의 확률 분포 차이 측정)
              metrics=['accuracy']) # 평가 지표로 정확도 사용

# Summary of the model
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x-in (InputLayer)           [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 28, 28, 5)         130       
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 14, 14, 5)         0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 14, 14, 5)         630       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 7, 7, 5)           0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 7, 7, 20)          2520

In [27]:
batch_size = 50
epochs = 50

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Evaluate the model
train_accuracy = model.evaluate(x_train, y_train, verbose=0)[1] # 손실값, 정확도 => 중에서 정확도를 출력하기 위해 [1] 인덱싱
print("Training accuracy: %.4f" % train_accuracy)


Epoch 1/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0379 - accuracy: 0.9902 - val_loss: 0.0875 - val_accuracy: 0.9840
Epoch 2/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0411 - accuracy: 0.9903 - val_loss: 0.0421 - val_accuracy: 0.9863
Epoch 3/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0434 - accuracy: 0.9901 - val_loss: 0.0457 - val_accuracy: 0.9867
Epoch 4/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0375 - accuracy: 0.9905 - val_loss: 0.0445 - val_accuracy: 0.9869
Epoch 5/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0410 - accuracy: 0.9905 - val_loss: 0.0414 - val_accuracy: 0.9865
Epoch 6/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0446 - accuracy: 0.9899 - val_loss: 0.0461 - val_accuracy: 0.9867
Epoch 7/50
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0345 - accuracy: 0.9908 - val_loss: 0.0412 -

2024-09-29 14:58:12.859287: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Training accuracy: 0.9937


In [28]:
test_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print("Test accuracy : %.4f" % test_accuracy) 

Test accuracy : 0.9881


# 모델 저장

In [30]:
# Save the model as an HDF5 file
model.save('./mnist_model.h5')  # HDF5 format